In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# Reading in data
data = pd.read_csv('technicals.csv')

In [3]:
# Setting index to datetime and removing bad columns
data = data.set_index('timestamp').drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1).copy()

In [4]:
# Creating buy and sell signals
data['call_buy'] = np.where((data['low'] <= data['lower_band']) & (data['time'] > '10:30:00'), True, False)
data['call_sell'] = np.where((data['high'] >= data['ema_10']) & (data['time'] > "10:30:00"), True, False)

data['put_buy'] = np.where((data['high'] >= data['upper_band']) & (data['time'] > '10:30:00'), True, False)
data['put_sell'] = np.where((data['low'] <= data['ema_10']) & (data['time'] > "10:30:00"), True, False)

In [5]:
# Removing times when buy and sell are at the same time
data = data[(data['call_buy'] & data['call_sell']) != True].copy()
data = data[(data['put_buy'] & data['put_sell']) != True].copy()

data = data[(data['call_buy'] | data['call_sell'] | data['put_buy'] | data['put_sell']) == True].copy()

In [6]:
# Initializing variabel for last eaction
last = None


# Looping through rows of data and saving index and row values
for index, row in data.iterrows():
    """
    Looping through all rows using logic to determine what vlaues each row should have based on the vlaue of the last action
    Results in data frame such that if call_buy = True than the next True value will be in the call_sell column
    and if put_buy = True than the next True value will be in the put_sell column
    """
    if row['call_buy']:
        if last == 'put_buy':
            data.at[index, 'call_buy'] = False
            data.at[index, 'call_sell'] = False
        
        elif last == 'call_buy':
            data.at[index, 'call_buy'] = False
            data.at[index, 'put_buy'] = False
            data.at[index, 'put_sell'] = False

        else:
            last = 'call_buy'
            data.at[index, 'put_sell'] = False

    if row['put_buy']:
        if last == 'put_buy':
            data.at[index, 'call_buy'] = False
            data.at[index, 'call_sell'] = False
            data.at[index, 'put_buy'] = False
        
        elif last == 'call_buy':
            data.at[index, 'put_sell'] = False
            data.at[index, 'put_buy'] = False

        else:
            last = 'put_buy'
            data.at[index, 'call_sell'] = False

    if row['call_sell']:
        if last == 'call_buy':
            last = 'call_sell'
            data.at[index, 'put_buy'] = False

        else:
            data.at[index, 'call_sell'] = False

    if row['put_sell']:
        if last == 'put_buy':
            last = 'put_sell'
            data.at[index, 'call_buy'] = False

        else:
            data.at[index, 'put_sell'] = False


In [7]:
data = data[(data['call_buy'] | data['call_sell'] | data['put_buy'] | data['put_sell']) == True].copy()

In [8]:
data.to_csv('signals.csv')
data = None